PyLDAvis now also support LDA application from scikit-learn. Let's take a look into this in more detail. We will be using 20new_groups dataset as provided by scikit-learn.

In [2]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
%matplotlib inline
import pyLDAvis
pyLDAvis.enable_notebook()

Next, we only fetch three categories from the news groups. If we set number of topics to 3 in LDA, we should get an intuitive result that measure intertopic distance map acrosss these categories.

In [24]:
cats = ['talk.religion.misc','comp.graphics', 'sci.med']
train = fetch_20newsgroups(subset='train',categories=cats,
                           shuffle=True,random_state=42)

# vect = CountVectorizer(stop_words='english')
lda = LatentDirichletAllocation(n_topics=3,random_state=123,max_iter=1500)

Finally, we use prepare method to visualize inside the notebook.

In [38]:
%%file ../pyLDAvis/sklearn.py
import pandas as pd
import funcy as fp
from . import prepare as vis_prepare

def _extract_data(docs, vect, lda):
  
    #LDA scikit-learn implementation seems to have buggy code.
    #Topic_term_dists and doc_topic_dists isn't accummulated to 1.
    #Hence norm function implemented to normalize the distributions.
    norm = lambda data: pd.DataFrame(data).div(data.sum(1),axis=0).values
    vected = vect.fit_transform(docs)
    doc_topic_dists = norm(lda.fit_transform(vected))
  
    return lda,vect, dict(
                      doc_lengths = docs.str.len(),
                      vocab = vect.get_feature_names(),
                      term_frequency = vected.sum(axis=0).tolist()[0],
                      topic_term_dists = norm(lda.components_),
                      doc_topic_dists = doc_topic_dists)

def prepare(docs, vect, lda, **kwargs):
    """Create Prepared Data from sklearn's vectorizer and Latent Dirichlet
    Application.

    Parameters
    ----------
    docs : Pandas Series.
        Documents to be passed as an input.
    vect : Scikit-Learn Vectorizer (CountVectorizer,TfIdfVectorizer).
        vectorizer to convert documents into matrix sparser
    lda  : sklearn.decomposition.LatentDirichletAllocation.
        Latent Dirichlet Allocation

    **kwargs: Keyword argument to be passed to pyLDAvis.prepare()


    Returns
    -------
    prepared_data : PreparedData
          the data structures used in the visualization


    Example
    --------
    For example usage please see this notebook:
    http://nbviewer.ipython.org/github/bmabey/pyLDAvis/blob/master/notebooks/Gensim%20Newsgroup.ipynb

    See
    ------
    See `pyLDAvis.prepare` for **kwargs.
    """
  
    opts = fp.merge(_extract_data(docs, vect, lda)[2], kwargs)

    return vis_prepare(**opts)

Writing ../pyLDAvis/sklearn.py


## Visualzing the model with pyLDAvis

In [14]:
prepare(pd.Series(train['data']),vect,lda)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      39.372307        1       1  0.086526  0.112715
2      33.205164        1       2 -0.159633  0.006499
0      27.422529        1       3  0.073107 -0.119214, topic_info=      Category         Freq          Term         Total  loglift  logprob
term                                                                     
13102  Default  6153.000000      graphics   6153.000000  30.0000  30.0000
12947  Default  5768.000000           god   5768.000000  29.0000  29.0000
14523  Default  5289.000000         image   5289.000000  28.0000  28.0000
18755  Default  3170.000000           msg   3170.000000  27.0000  27.0000
15623  Default  4154.000000         jesus   4154.000000  26.0000  26.0000
21163  Default  4217.000000          pitt   4217.000000  25.0000  25.0000
2897   Default  3210.000000           ___   3210.000000  24.0000  24.0000
11877  Default  3087.000000          file   3087.000000  23.0000  23.0000
15777  Default  2635.000000          jpeg   2635.000000  22.0000  22.0000
11881  Default  2576.000000         files   2576.000000  21.0000  21.0000
7110   Default  2944.000000     christian   2944.000000  20.0000  20.0000
5062   Default  2892.000000         banks   2892.000000  19.0000  19.0000
14533  Default  2433.000000        images   2433.000000  18.0000  18.0000
12151  Default  2456.000000          food   2456.000000  17.0000  17.0000
13634  Default  2696.000000        health   2696.000000  16.0000  16.0000
13003  Default  3067.000000        gordon   3067.000000  15.0000  15.0000
12639  Default  2764.000000           geb   2764.000000  14.0000  14.0000
12434  Default  2226.000000           ftp   2226.000000  13.0000  13.0000
1694   Default  2146.000000            3d   2146.000000  12.0000  12.0000
9756   Default  2020.000000       disease   2020.000000  11.0000  11.0000
20620  Default  1784.000000      patients   1784.000000  10.0000  10.0000
17891  Default  2674.000000       medical   2674.000000   9.0000   9.0000
12791  Default  1974.000000           gif   1974.000000   8.0000   8.0000
12213  Default  1967.000000        format   1967.000000   7.0000   7.0000
25280  Default  3362.000000      software   3362.000000   6.0000   6.0000
5441   Default  2301.000000         bible   2301.000000   5.0000   5.0000
24050  Default  2199.000000       sandvik   2199.000000   4.0000   4.0000
10369  Default  1524.000000          dyer   1524.000000   3.0000   3.0000
7533   Default  2215.000000         color   2215.000000   2.0000   2.0000
29434  Default  1716.000000       windows   1716.000000   1.0000   1.0000
...        ...          ...           ...           ...      ...      ...
27595   Topic3  1201.863877     treatment   1390.085776   1.1483  -6.5073
29159   Topic3  1258.695994         water   1514.729654   1.1086  -6.4611
9990    Topic3  1569.586052        doctor   2024.113467   1.0395  -6.2403
6670    Topic3  1300.558045         cause   1665.498040   1.0465  -6.4284
23581   Topic3   676.914511          risk    724.338061   1.2261  -7.0814
9548    Topic3   937.604806          diet   1169.541851   1.0728  -6.7556
17897   Topic3  1087.238550      medicine   1461.017469   0.9983  -6.6075
7561    Topic3  4949.271723           com  18522.805939  -0.0260  -5.0919
20375   Topic3  1102.824193          pain   1625.031544   0.9062  -6.5933
26086   Topic3  4473.758363       subject  17816.572878  -0.0881  -5.1929
10519   Topic3  5695.149605           edu  31834.939165  -0.4271  -4.9515
20062   Topic3  3960.281217  organization  16497.712480  -0.1331  -5.3148
16896   Topic3  3923.100827         lines  16887.546702  -0.1659  -5.3243
28394   Topic3  2139.808284           use   5920.816195   0.2760  -5.9304
4545    Topic3  2904.549501       article  11213.372111  -0.0570  -5.6249
29659   Topic3  2729.958772        writes  11124.718518  -0.1111  -5.6869
20777   Topic3  2321.627731        people   8371.661628   0.0

Alternatively, you can also have LDA model that have sklearn's API, for example https://github.com/ariddell/lda.

In [15]:
from lda import LDA

lda_b = LDA(n_topics=len(cats),n_iter=1500,random_state=123)

In [17]:
prepare(pd.Series(train['data']),vect,lda_b)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      46.645322        1       1  0.055933  0.249211
0      28.982198        1       2 -0.270468 -0.081495
1      24.372480        1       3  0.214535 -0.167716, topic_info=      Category          Freq          Term         Total  loglift  logprob
term                                                                      
13102  Default   6538.000000      graphics   6538.000000  30.0000  30.0000
14523  Default   6056.000000         image   6056.000000  29.0000  29.0000
10519  Default  35370.000000           edu  35370.000000  28.0000  28.0000
12947  Default   6238.000000           god   6238.000000  27.0000  27.0000
11877  Default   3462.000000          file   3462.000000  26.0000  26.0000
29659  Default  12138.000000        writes  12138.000000  25.0000  25.0000
24277  Default   5219.000000       science   5219.000000  24.0000  24.0000
27093  Default   6753.000000         think   6753.000000  23.0000  23.0000
7561   Default  21029.000000           com  21029.000000  22.0000  22.0000
15777  Default   3001.000000          jpeg   3001.000000  21.0000  21.0000
21929  Default   3582.000000       program   3582.000000  20.0000  20.0000
11881  Default   2990.000000         files   2990.000000  19.0000  19.0000
15623  Default   4637.000000         jesus   4637.000000  18.0000  18.0000
21163  Default   4507.000000          pitt   4507.000000  17.0000  17.0000
14533  Default   2701.000000        images   2701.000000  16.0000  16.0000
8899   Default   3333.000000          data   3333.000000  15.0000  15.0000
12434  Default   2583.000000           ftp   2583.000000  14.0000  14.0000
7533   Default   2562.000000         color   2562.000000  13.0000  13.0000
25280  Default   3649.000000      software   3649.000000  12.0000  12.0000
18755  Default   3724.000000           msg   3724.000000  11.0000  11.0000
4863   Default   3515.000000     available   3515.000000  10.0000  10.0000
4545   Default  12155.000000       article  12155.000000   9.0000   9.0000
1694   Default   2240.000000            3d   2240.000000   8.0000   8.0000
5956   Default   1962.000000         brian   1962.000000   7.0000   7.0000
12791  Default   2229.000000           gif   2229.000000   6.0000   6.0000
12213  Default   2219.000000        format   2219.000000   5.0000   5.0000
24148  Default   4486.000000           say   4486.000000   4.0000   4.0000
13003  Default   3427.000000        gordon   3427.000000   3.0000   3.0000
18607  Default   1800.000000      morality   1800.000000   2.0000   2.0000
21351  Default   1928.000000        points   1928.000000   1.0000   1.0000
...        ...           ...           ...           ...      ...      ...
8737    Topic3    554.006919          cwru    554.232708   1.4113  -7.1638
28442   Topic3    980.964970        utexas    991.909496   1.4006  -6.5925
21351   Topic3   1857.959885        points   1928.204954   1.3746  -5.9538
24277   Topic3   3969.671324       science   5219.276868   1.1380  -5.1946
18135   Topic3   1338.686580       michael   1553.287125   1.2630  -6.2815
10519   Topic3  14770.556064           edu  35370.831720   0.5385  -3.8806
27093   Topic3   3658.107341         think   6753.821501   0.7986  -5.2763
29659   Topic3   5631.345900        writes  12138.601541   0.6437  -4.8449
7561    Topic3   8366.185305           com  21029.329368   0.4900  -4.4490
19387   Topic3   3404.240392          nntp   6802.209729   0.7195  -5.3482
4545    Topic3   4950.520900       article  12155.884443   0.5134  -4.9737
20062   Topic3   6496.801408  organization  18048.367142   0.3900  -4.7019
26086   Topic3   6635.274289       subject  19224.574164   0.3479  -4.6808
14111   Topic3   3381.161579          host   7089.081513   0.6714  -5.3550
24148   Topic3   2434.930223           say   4486.954817   0.8005  -5.6833
16896   Topic3   6104.461577         lines  18497.100746   0.3031  -4.7642
21522   Topi

Note that you can get the vectorizer and LDA that has been learned on the data, by using `_extract_data` method.

In [31]:
lda_fit,vect_fit, prepared = _extract_data(pd.Series(train['data']),vect,lda_b)

In [32]:
vect_fit

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [35]:
lda_fit